<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Дерево-решений" data-toc-modified-id="Дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Дерево решений</a></span></li><li><span><a href="#Случайный-лес" data-toc-modified-id="Случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Случайный лес</a></span></li><li><span><a href="#Градиентный-бустинг-CatBoost" data-toc-modified-id="Градиентный-бустинг-CatBoost-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Градиентный бустинг CatBoost</a></span></li><li><span><a href="#Градиентный-бустинг-Light-GBM" data-toc-modified-id="Градиентный-бустинг-Light-GBM-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Градиентный бустинг Light GBM</a></span></li><li><span><a href="#Проверка-модели" data-toc-modified-id="Проверка-модели-2.6"><span class="toc-item-num">2.6&nbsp;&nbsp;</span>Проверка модели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект по модерации токсичных комментариев

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые библиотеки и инструменты для работы с данными.

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import warnings

from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV
from sklearn.metrics import f1_score

from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier

from pymystem3 import Mystem

Отключим предупреждения.

In [2]:
warnings.filterwarnings('ignore')

Сохраним файл в переменную df и рассмотрим первые пять строк таблицы.

In [3]:
df = pd.read_csv('toxic_comments.csv')
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


Изучим основную информацию о таблице.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Удалим столбец "Unnamed: 0", т.к. он не несет смысловой нагрузки.

In [5]:
df = df.drop('Unnamed: 0', axis=1)

Проверим наличие пропусков в столбцах и дубликатов.

In [6]:
print('Количество пропусков встолбце text:', df['text'].isna().sum())
print('Количество пропусков встолбце toxic:', df['toxic'].isna().sum())
print('Количество полных дубликатов:', df.duplicated().sum())

Количество пропусков встолбце text: 0
Количество пропусков встолбце toxic: 0
Количество полных дубликатов: 0


Рассмотрим соотношение позитивных и негативных комментариев.

In [7]:
df['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

В выборке наблюдается сильная несбалансированность классов: позитивных комментариев в 9 раз больше, чем негативных.

Для дальнейшей работы с текстом очистим его от лишних символов, оставим только английские символы и пробелы.

In [8]:
df['clear_text'] = df['text'].apply(lambda x: " ".join(re.sub(r'[^a-zA-Z ]', ' ', x).split()))

Напишем функцию для лемматизации текста и применим ее к очищенным комментариям.

In [10]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize_text(sentence):
    lemmatizer = WordNetLemmatizer()
    word_list = nltk.word_tokenize(sentence)
    return ' '.join([lemmatizer.lemmatize(w.lower(), get_wordnet_pos(w)) for w in nltk.word_tokenize(sentence)])

In [12]:
nltk.download('averaged_perceptron_tagger')
df['lemm_text'] = df['clear_text'].apply(lemmatize_text)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

Еще раз взглянем на первые пять строк таблицы.

In [11]:
df.head()

,text,toxic,clear_text,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...,explanation why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D aww He matches this background colour I m se...,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I m really not trying to edit war It s...,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can t make any real suggestions on impr...,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...,you sir be my hero any chance you remember wha...


Данные готовы для подготовки к обучению моделей.

Сохраним признак и целевой признак в отдельные переменные.

In [12]:
features = df['lemm_text']
target = df['toxic']

Разделим данные на обучающую и тестовую выборки в соотношении 3:1.

In [13]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345, stratify=target)

Векторизируем текст для обучения моделей. Обучим векторизатор на обучающей выборке, затем преобразуем обучающую и тестовую выборки.

In [14]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
tf_idf = TfidfVectorizer(stop_words=stopwords, analyzer='word', min_df=10).fit(features_train)
features_train = tf_idf.transform(features_train)
features_test = tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Приступим к обучению моделей.

## Обучение

In [15]:
min_f1 = 0.75 # минимальный целевой уровень метрики f1

### Логистическая регрессия

In [16]:
%%time

model_lr = LogisticRegression(random_state=12345, solver='saga', C=3, class_weight='balanced')
scores_lr = cross_val_score(model_lr, features_train, target_train, scoring='f1', cv=3)
f1_lr = scores_lr.mean()

print('F1 логистической регрессии:', f1_lr)
if f1_lr < min_f1:
    print('F1 ниже целевого уровня', '\n')
else:
    print('F1 выше целевого уровня', '\n')

F1 логистической регрессии: 0.7398196365352819
F1 ниже целевого уровня 

CPU times: user 17.8 s, sys: 22.6 ms, total: 17.8 s
Wall time: 17.8 s


### Дерево решений

In [17]:
%%time

params_dtr = {'max_depth': range(2, 7)}

model_dtc = DecisionTreeClassifier(random_state=12345)
gscv_dtc = GridSearchCV(model_dtc, params_dtr, scoring='f1', cv=3, n_jobs=-1)
gscv_dtc.fit(features_train, target_train)
f1_dtc = gscv_dtc.best_score_

print('F1 дерева решений:', f1_dtc)
print('Лучшие гиперпараметры:', gscv_dtc.best_params_)
if f1_dtc < min_f1:
    print('F1 ниже целевого уровня', '\n')
else:
    print('F1 выше целевого уровня', '\n')

F1 дерева решений: 0.5514246374113291
Лучшие гиперпараметры: {'max_depth': 6}
F1 ниже целевого уровня 

CPU times: user 18.4 s, sys: 224 ms, total: 18.6 s
Wall time: 18.7 s


### Случайный лес

In [18]:
%%time

params_rfc = {
    'n_estimators': range(10, 101, 10),
    'max_depth': range(2, 7)
}

model_rfc = RandomForestClassifier(random_state=12345, class_weight='balanced')
gscv_rfc = GridSearchCV(model_rfc, params_rfc, scoring='f1', cv=3, n_jobs=-1)
gscv_rfc.fit(features_train, target_train)
f1_rfc = gscv_rfc.best_score_

print('F1 случайного леса:', f1_rfc)
print('Лучшие гиперпараметры:', gscv_rfc.best_params_)
if f1_rfc < min_f1:
    print('F1 ниже целевого уровня', '\n')
else:
    print('F1 выше целевого уровня', '\n')

F1 случайного леса: 0.36514151680912627
Лучшие гиперпараметры: {'max_depth': 6, 'n_estimators': 100}
F1 ниже целевого уровня 

CPU times: user 2min 58s, sys: 2.2 s, total: 3min
Wall time: 3min


### Градиентный бустинг CatBoost

In [19]:
%%time

model_cb = CatBoostClassifier(eval_metric='F1', iterations=100, random_state=12345)
scores_cb = cross_val_score(model_cb, features_train, target_train, fit_params={'verbose': False}, scoring='f1', cv=3)
f1_cb = scores_cb.mean()

print('F1 градиентного бустинга CatBoost:', f1_cb)
if f1_cb < min_f1:
    print('F1 ниже целевого уровня', '\n')
else:
    print('F1 выше целевого уровня', '\n')

F1 градиентного бустинга CatBoost: 0.7336233298750701
F1 ниже целевого уровня 

CPU times: user 6min 6s, sys: 2.11 s, total: 6min 8s
Wall time: 6min 12s


### Градиентный бустинг Light GBM

In [20]:
%%time

model_lgbm = LGBMClassifier(metric='f1', max_depth=7, random_state=12345)
scores_lgbm = cross_val_score(model_lgbm, features_train, target_train, scoring='f1', cv=3)
f1_lgbm = scores_lgbm.mean()

print('F1 градиентного бустинга Light GBM:', f1_lgbm)
if f1_lgbm < min_f1:
    print('F1 ниже целевого уровня', '\n')
else:
    print('F1 выше целевого уровня', '\n')

F1 градиентного бустинга Light GBM: 0.6645834495832695
F1 ниже целевого уровня 

CPU times: user 33min 7s, sys: 20.6 s, total: 33min 28s
Wall time: 33min 38s


### Проверка модели

На обучающей выборке не удалось добиться целевого уровня f1.
Самые высокие результаты метрики f1 показали логистическая регрессия и градиентный бустинг CatBoost.  
Однако у логистической регрессии также самое быстрое время обучения и предсказания.  
Проверим ее на тестовой выборке.

In [21]:
model_lr.fit(features_train, target_train)
predicted = model_lr.predict(features_test)
f1_lr_test = f1_score(target_test, predicted)
f1_lr_test

0.7589871128193534

На логистической регрессии удалось достичь целевого значения метрики f1.

## Выводы

В рамках проекта были обработаны размеченные данные с комментариями пользователей к описанию товаров.  

Перед обучением моделей данные были очищены от лишних символов, лемматизированы и векторизованы.  

В данных присутствует дисбаланс классов: позитивных комментариев в 9 раз больше, чем негативных.  

Были обучены 5 моделей, при использовании кросс-валидации удалось достичь следующих значений метрики F1 на обучающей выборке:

In [23]:
models = ['Линейная регрессия', 'Дерево решений', 'Случайный лес', 'Градиентный бустинг CatBoost', 'Градиентный бустинг Light GBM']
results = pd.Series([f1_lr, f1_dtc, f1_rfc, f1_cb, f1_lgbm], index=models)
results

Линейная регрессия               0.739820
Дерево решений                   0.551425
Случайный лес                    0.365142
Градиентный бустинг CatBoost     0.733623
Градиентный бустинг Light GBM    0.664583
dtype: float64

Ни на одной из моделей не удалось достичь целевого значения метрики F1 (0.75) на обучающей выборке. Однако логистическая регрессия и градиентный бустинг CatBoost показали самые высокие результаты метрики. С учетом скорости обучения и предсказания для проверки на тестовой выборке была выбрана логистическая регрессия.  

Получено значение F1 на тестовой выборке равное 0.7589871128193534.

Так как логистическая регрессия на тестовой выборке показала значение метрики F1 выше целевого уровня, данную модель рекомендуется использовать для выявления токсичных комментариев.